The Turán number of $C_5^-$
==============================

This notebook contains calculations for the Turán number of $C_5^-$. To run these calculations, use the modified version of sage from
https://github.com/bodnalev/sage

As the blowup of $K_4^-$ contains $C_5^-$, we can additionally assume 
that we work in $K_4^-$-free structures. 

1. The first cell sets up the combinatorial theory of $C_5^-$ and 
$K_4^-$-free 3-graphs (called TGp). In addition, it sets up the 
combinatorial theory on the same 3-graphs with vertices partitioned 
into 3 parts using an additional 2-ary relation indicating different parts (called CTGp)

2. The second cell performs the basic calculation of upper bounding edges 
in the theory for Proposition 3.1. It gives the $\alpha_{3, 1}$ = a31 = 126373441/504000000 ~= 0.250740954365079... upper bound.
The certificate proving the claim is saved to the file "proposition_3_1.pickle".

3. The next cell lower bounds the max-cut ration (k222) for Proposition 3.2.
Note at the optimal construction this value is exactly 25/121 ~= 0.19834710743801652892...
The exact lower bound provided is $\alpha_{3, 2}$ = a32 = 1607168566087/8282009829376 ~= 0.194055380179148...
The certificate proving the claim is saved to the file "proposition_3_2.pickle".

5. The following cell works in the 3 partitioned theory.
It uses the lower bound b33=0.19 on the edges in the correct parts, where $\beta_{3, 3}$ = b33 < a32 from the previous calculation.
The calculations provide the precise density bound that there are less bad
edges than missing edges asymptotically on the top
level. Here bad and missing is defined compared to the expected construction.
The certificate proving this claim is saved to the file "proposition_3_3.pickle".

6. The next cell establishes the theory for 3 colored graphs appearing as
a link of a vertex. The patterns excluded all would result in a $C_5^-$
assuming that all the 3-edges between the three parts are present.

7. Then the calculation on the link graph is performed.
The calculations provide the precise density bound that there are less bad
edges than missing edges asymptotically. The
certificate proving this claim is saved to the file "proposition_3_4.pickle".

8. The first 6 cells perform the entire calculation from scratch. If one
only wants to verify that any of the certificates are indeed correct, it is enough
to run the corresponding cell from the final 4. For each step above, it loads the generated
certificates and verifies that the matrices are indeed positive semidefinite
and that the bound they prove is exactly as claimed.

In [1]:
### Theory for 3-graphs, with k4m and c5m excluded
k4m = ThreeGraphTheory.pattern(4, edges=[[0, 1, 2], [0, 1, 3], [0, 2, 3]])
c5m = ThreeGraphTheory.pattern(5, edges=[[0, 1, 2], [1, 2, 3], [2, 3, 4], [3, 4, 0]])
ThreeGraphTheory.exclude([k4m, c5m])
TGp = ThreeGraphTheory


# Graphs representing a three-partition
P = Theory("3Partition", relation_name="part", arity=2, is_ordered=False)
P.exclude([P(3, part=[[0, 1]]), P(4, part=[[0, 1], [0, 2], [0, 3], [1, 2], [1, 3], [2, 3]])])
ThreePartitionedThreeGraphTheory = combine("3PartitionNoC5m", TGp, P)
CTGp = ThreePartitionedThreeGraphTheory

In [3]:
### This part just gives a standard upper bound on the number of edges without C5- and K4- (Proposition 3.1)

a31 = TGp.optimize(TGp(3, edges=[[0, 1, 2]]), 7,  exact=True, 
                   file="certificates/proposition_3_1", 
                   denom=1024*3125, printlevel=0
                  )
print("The initial upper bound on the Turan density from Proposition 3.1 is {} ~= {}".format(a31, a31.n()))

The initial upper bound on the Turan density from Proposition 3.1 is 401181/1600000 ~= 0.250738125000000


In [4]:
### This code that minimizes the max-cut ratio K222 (Proposition 3.2)

# beta_{3, 2} constant
b32 = 1/4 - 1/100000
edge = TGp(3, edges=[[0, 1, 2]])

# First the typed f222 is constructed
f222 = TGp.pattern(6, ftype=[0, 1, 2], 
                   edges=[[0, 1, 2], [3, 4, 5], [0, 1, 5], [0, 2, 4], [1, 2, 3]])
# Then k222 is the projection of f222, this takes care of the automorphisms
k222 = f222.project()

gamma = TGp.optimize(k222, 7, maximize=False, positives=[edge - b32], exact=True, 
                     file="certificates/proposition_3_2", 
                     denom=1024*1024, printlevel=0
                    )
a32 = gamma / a31

print("The max-cut ratio returned by Proposition 3.2 is at least {} ~= {}".format(a32, a32.n()))

The max-cut ratio returned by Proposition 3.2 is at least 1701468433/8763932672 ~= 0.194144398032181


In [7]:
### This is the code that performs the calculations on the theory with three partition (Proposition 3.3)

# edge with (C)orrect partition
C = CTGp(3, edges=[[0, 1, 2]], part=[[0, 1], [0, 2], [1, 2]])
# edge with (C)orrect partition (p)ointed
Cp = CTGp(3, edges=[[0, 1, 2]], part=[[0, 1], [0, 2], [1, 2]], ftype=[0])

# edge with (B)ad partition
B = CTGp(3, edges=[[0, 1, 2]], part=[[0, 1], [1, 2]])

# edge with (B)ad partition (p)ointed 
Bp = CTGp(3, edges=[[0, 1, 2]], part=[[0, 1], [1, 2]], ftype=[0])

# (M)issing edge with good partition
M = CTGp(3, edges=[], part=[[0, 1], [0, 2], [1, 2]])

# positivity assumptions:
# each point, good edges are more than bad edges 
# (divided by two due to the symmetry between the parts)
# edge density is larger than b33 (which is smaller than a32)
b33 = 19/100
assums = [Cp - Bp/2, C - b33]

# optimal construction and its derivatives
symbolic_constr = CTGp.blowup_construction(6, ["X0", "X1", "X2"], edges=[[0, 1, 2]], 
                                           part=[[0, 1], [0, 2], [1, 2]]).set_sum()
ders = symbolic_constr.derivatives([1/3, 1/3])

# bad is less than missing, proven by (B)ad (M)inum (M)issing is at most 0.
CTGp.optimize(B - M*(99/100), 6, positives=assums, exact=True, 
              construction=ders, file="certificates/proposition_3_3", 
              denom=1024*16, slack_threshold=1e-6, kernel_denom=2**20, printlevel=0
             )

0

In [1]:
### Theory for 3-colored 2-graphs, the colors are not interchangeable here
Asymm3ColorTheory = combine("Asymm3Colors", Color0, Color1, Color2, 
                            symmetries=NoSymmetry)
# Again, force the colors to be disjoint
Asymm3ColorTheory.exclude([
    Asymm3ColorTheory(1), 
    Asymm3ColorTheory(1, C0=[0], C1=[0]), 
    Asymm3ColorTheory(1, C0=[0], C2=[0]), 
    Asymm3ColorTheory(1, C1=[0], C2=[0]),
    Asymm3ColorTheory(1, C0=[0], C1=[0], C2=[0])
])
ColoredLinkGraphTheory = combine("ColoredLinkGraph", GraphTheory, Asymm3ColorTheory)
CLGT = ColoredLinkGraphTheory
# These non-induced patterns guarantee the theory 
# represents a 3 colored c5m-free 3-graph's link.
CLGT.exclude([
    CLGT.pattern(4, edges=[[0, 1], [2, 3]], C0=[0], C1=[1, 2], C2=[3]),
    CLGT.pattern(4, edges=[[0, 1], [2, 3]], C0=[1, 2], C1=[0], C2=[3]),
    CLGT.pattern(4, edges=[[0, 1], [2, 3]], C0=[0], C1=[3], C2=[1, 2]),
    
    CLGT.pattern(3, edges=[[0, 1], [1, 2]], C0=[0], C1=[1], C2=[2]),
    CLGT.pattern(3, edges=[[0, 1], [1, 2]], C0=[1], C1=[0], C2=[2]),
    CLGT.pattern(3, edges=[[0, 1], [1, 2]], C0=[0], C1=[2], C2=[1]),

    CLGT.pattern(3, edges=[[0, 1], [1, 2]], C0=[0, 1], C1=[2], C2=[]),
    CLGT.pattern(3, edges=[[0, 1], [1, 2]], C0=[0, 1], C1=[], C2=[2]),
    CLGT.pattern(3, edges=[[0, 1], [1, 2]], C0=[2], C1=[0, 1], C2=[]),
    CLGT.pattern(3, edges=[[0, 1], [1, 2]], C0=[], C1=[0, 1], C2=[2]),
    CLGT.pattern(3, edges=[[0, 1], [1, 2]], C0=[2], C1=[], C2=[0, 1]),
    CLGT.pattern(3, edges=[[0, 1], [1, 2]], C0=[], C1=[2], C2=[0, 1])
])

In [2]:
### Vertex stability part (Proposition 3.4)

edge_00 = CLGT(2, edges=[[0, 1]], C0=[0, 1], C1=[], C2=[])
edge_11 = CLGT(2, edges=[[0, 1]], C0=[], C1=[0, 1], C2=[])
edge_22 = CLGT(2, edges=[[0, 1]], C0=[], C1=[], C2=[0, 1])
edge_01 = CLGT(2, edges=[[0, 1]], C0=[0], C1=[1], C2=[])
edge_12 = CLGT(2, edges=[[0, 1]], C0=[], C1=[0], C2=[1])
edge_02 = CLGT(2, edges=[[0, 1]], C0=[0], C1=[], C2=[1])

point0 = CLGT(1, C0 = [0])
point1 = CLGT(1, C1 = [0])
point2 = CLGT(1, C2 = [0])

positives = [
    edge_12 - edge_01, 
    edge_12 - edge_02,
    edge_01 + edge_02 + edge_12 - 1/8,
    point0 - 1/4, 
    point1 - 1/4, 
    point2 - 1/4
]

# (M)issing edge with good colors
M = CLGT(2, edges=[], C0=[], C1=[0], C2=[1]) 

# edges with (B)ad colors 
B = sum([
    CLGT(2, edges=[[0, 1]], C0=[0], C1=[1], C2=[]),
    CLGT(2, edges=[[0, 1]], C0=[0], C1=[], C2=[1]),
    CLGT(2, edges=[[0, 1]], C0=[], C1=[0, 1], C2=[]),
    CLGT(2, edges=[[0, 1]], C0=[], C1=[], C2=[0, 1])
])

CLGT.optimize(B - M*9/10, 5, positives = positives, exact=True, file="certificates/proposition_3_4_alter", construction=False)

Base flags generated, their number is 450
The relevant ftypes are constructed, their number is 43
Block sizes before symmetric/asymmetric change is applied: [28, 28, 28, 24, 15, 15, 24, 15, 12, 15, 15, 15, 24, 12, 8, 9, 9, 8, 12, 8, 9, 6, 6, 6, 8, 9, 9, 8, 9, 8, 12, 10, 7, 7, 10, 7, 7, 7, 7, 10, 10, 10, 10]


Done with mult table for Ftype on 3 points with edges=(01 02 12), C0=(0 1 2), C1=(), C2=(): : 43it [00:00, 364.39it/s]


Tables finished


Done with positivity constraint 5: 100%|██████████| 6/6 [00:00<00:00, 52.40it/s]


Constraints finished
Running sdp without construction. Used block sizes are [28, 28, 28, 12, 12, 12, 3, 12, 3, 12, 12, 12, 3, 12, 12, 3, 12, 3, 12, 3, 12, 12, 10, 2, 8, 8, 1, 8, 1, 8, 10, 2, 8, 8, 1, 6, 6, 6, 8, 8, 1, 8, 1, 8, 8, 1, 8, 10, 2, 8, 2, 6, 1, 6, 1, 8, 2, 6, 1, 6, 1, 6, 1, 6, 1, 8, 2, 6, 4, 6, 4, 6, 4, -450, -527]
CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 1.00e+00 Pobj: -4.9813379e+01 Ad: 7.04e-01 Dobj:  7.9174963e+00 
Iter:  2 Ap: 1.00e+00 Pobj: -5.1678202e+01 Ad: 9.53e-01 Dobj: -8.8343906e-02 
Iter:  3 Ap: 1.00e+00 Pobj: -5.2645196e+01 Ad: 9.47e-01 Dobj: -7.5268893e-02 
Iter:  4 Ap: 8.96e-01 Pobj: -4.5632845e+01 Ad: 8.46e-01 Dobj: -4.9953745e-02 
Iter:  5 Ap: 8.93e-01 Pobj: -2.6162277e+01 Ad: 5.30e-01 Dobj: -1.0236458e-02 
Iter:  6 Ap: 2.76e-01 Pobj: -2.4347561e+01 Ad: 4.56e-01 Dobj:  1.9157059e-02 
Iter:  7 Ap: 9.85e-01 Pobj: -1.3978511e+01 Ad: 5.71e-01 Dobj:  4.2884754e-02 
Iter:  8 Ap: 1.00e+00 Pobj: -7.8002970

100%|██████████████████████████████████████████| 73/73 [00:00<00:00, 786.71it/s]


Calculating resulting bound


100%|███████████████████████████████████████████| 43/43 [00:11<00:00,  3.74it/s]


The rounded result is 0


0

In [4]:
print("\n".join(map(str, enumerate(-vector([1, 2, 3, 4, 5, 11, 12])))))

(0, -1)
(1, -2)
(2, -3)
(3, -4)
(4, -5)
(5, -11)
(6, -12)


Verify the certificates produced
===============================

Can be run without running the above cells. Note however that the 
initial call to these cells might take longer, due to the calculation of
the multiplication tables. Once that is complete and stored, these cells 
verify the results fairly quickly.

In [7]:
### Verify Proposition 3.1

k4m = ThreeGraphTheory.pattern(4, edges=[[0, 1, 2], [0, 1, 3], [0, 2, 3]])
c5m = ThreeGraphTheory.pattern(5, edges=[[0, 1, 2], [1, 2, 3], [2, 3, 4], [3, 4, 0]])
ThreeGraphTheory.exclude([k4m, c5m])
TGp = ThreeGraphTheory
TGp.verify("certificates/proposition_3_1", TGp(3, edges=[[0, 1, 2]]), 7)

Checking X matrices


12it [02:24, 12.03s/it]


Solution matrices are all positive semidefinite, linear coefficients are all non-negative
Calculating multiplication tables


12it [00:01,  7.58it/s]


Done calculating linear constraints
Calculating the bound provided by the certificate


12it [03:02, 15.23s/it]


The solution is valid, it proves the bound 63186647/252000000


63186647/252000000

In [8]:
### Verify Proposition 3.2

k4m = ThreeGraphTheory.pattern(4, edges=[[0, 1, 2], [0, 1, 3], [0, 2, 3]])
c5m = ThreeGraphTheory.pattern(5, edges=[[0, 1, 2], [1, 2, 3], [2, 3, 4], [3, 4, 0]])
ThreeGraphTheory.exclude([k4m, c5m])
TGp = ThreeGraphTheory

b32 = 1/4 - 1/100000
edge = TGp(3, edges=[[0, 1, 2]])
f222 = TGp.pattern(6, ftype=[0, 1, 2], edges=[[0, 1, 2], [3, 4, 5], [0, 1, 5], [0, 2, 4], [1, 2, 3]]).afae()
k222 = f222.project()

TGp.verify("certificates/proposition_3_2", k222, 7, maximize=False, positives=[edge - b32])

Checking X matrices


12it [02:01, 10.09s/it]


Solution matrices are all positive semidefinite, linear coefficients are all non-negative
Calculating multiplication tables


12it [00:01,  7.70it/s]


Done with positivity constraint 0
Done calculating linear constraints
Calculating the bound provided by the certificate


12it [03:14, 16.19s/it]


The solution is valid, it proves the bound 803712917869/16515072000000


803712917869/16515072000000

In [9]:
### Verify Proposition 3.3

k4m = ThreeGraphTheory.pattern(4, edges=[[0, 1, 2], [0, 1, 3], [0, 2, 3]])
c5m = ThreeGraphTheory.pattern(5, edges=[[0, 1, 2], [1, 2, 3], [2, 3, 4], [3, 4, 0]])
ThreeGraphTheory.exclude([k4m, c5m])
TGp = ThreeGraphTheory
P = Theory("3Partition", relation_name="part", arity=2, is_ordered=False)
P.exclude([P(3, part=[[0, 1]]), P(4, part=[[0, 1], [0, 2], [0, 3], [1, 2], [1, 3], [2, 3]])])
ThreePartitionedThreeGraphTheory = combine("3PartitionNoC5m", TGp, P)
CTGp = ThreePartitionedThreeGraphTheory

C = CTGp(3, edges=[[0, 1, 2]], part=[[0, 1], [0, 2], [1, 2]])
Cp = CTGp(3, edges=[[0, 1, 2]], part=[[0, 1], [0, 2], [1, 2]], ftype=[0])
B = CTGp(3, edges=[[0, 1, 2]], part=[[0, 1], [1, 2]])
Bp = CTGp(3, edges=[[0, 1, 2]], part=[[0, 1], [1, 2]], ftype=[0])
M = CTGp(3, edges=[], part=[[0, 1], [0, 2], [1, 2]])
b33 = 19/100
assums = [Cp - Bp/2, C - b33]
CTGp.verify("certificates/proposition_3_3", B + (-99/100)*M, 6, positives=assums)

Checking X matrices


20it [00:04,  4.75it/s]


Solution matrices are all positive semidefinite, linear coefficients are all non-negative
Calculating multiplication tables


20it [00:01, 17.51it/s]


Done with positivity constraint 0
Done with positivity constraint 1
Done calculating linear constraints
Calculating the bound provided by the certificate


20it [02:36,  7.82s/it]


The solution is valid, it proves the bound 0


0

In [4]:
### Verify Proposition 3.4

Asymm3ColorTheory = combine("Asymm3Colors", Color0, Color1, Color2, symmetries=NoSymmetry)
Asymm3ColorTheory.exclude([
    Asymm3ColorTheory(1), 
    Asymm3ColorTheory(1, C0=[0], C1=[0]), 
    Asymm3ColorTheory(1, C0=[0], C2=[0]), 
    Asymm3ColorTheory(1, C1=[0], C2=[0]),
    Asymm3ColorTheory(1, C0=[0], C1=[0], C2=[0])
])
ColoredLinkGraphTheory = combine("ColoredLinkGraph", GraphTheory, Asymm3ColorTheory)
CLGT = ColoredLinkGraphTheory
CLGT.exclude([
    CLGT.pattern(4, edges=[[0, 1], [2, 3]], C0=[0], C1=[1, 2], C2=[3]),
    CLGT.pattern(4, edges=[[0, 1], [2, 3]], C0=[1, 2], C1=[0], C2=[3]),
    CLGT.pattern(4, edges=[[0, 1], [2, 3]], C0=[0], C1=[3], C2=[1, 2]),
    
    CLGT.pattern(3, edges=[[0, 1], [1, 2]], C0=[0], C1=[1], C2=[2]),
    CLGT.pattern(3, edges=[[0, 1], [1, 2]], C0=[1], C1=[0], C2=[2]),
    CLGT.pattern(3, edges=[[0, 1], [1, 2]], C0=[0], C1=[2], C2=[1]),

    CLGT.pattern(3, edges=[[0, 1], [1, 2]], C0=[0, 1], C1=[2], C2=[]),
    CLGT.pattern(3, edges=[[0, 1], [1, 2]], C0=[0, 1], C1=[], C2=[2]),
    CLGT.pattern(3, edges=[[0, 1], [1, 2]], C0=[2], C1=[0, 1], C2=[]),
    CLGT.pattern(3, edges=[[0, 1], [1, 2]], C0=[], C1=[0, 1], C2=[2]),
    CLGT.pattern(3, edges=[[0, 1], [1, 2]], C0=[2], C1=[], C2=[0, 1]),
    CLGT.pattern(3, edges=[[0, 1], [1, 2]], C0=[], C1=[2], C2=[0, 1])
])

edge_00 = CLGT(2, edges=[[0, 1]], C0=[0, 1], C1=[], C2=[])
edge_11 = CLGT(2, edges=[[0, 1]], C0=[], C1=[0, 1], C2=[])
edge_22 = CLGT(2, edges=[[0, 1]], C0=[], C1=[], C2=[0, 1])
edge_01 = CLGT(2, edges=[[0, 1]], C0=[0], C1=[1], C2=[])
edge_12 = CLGT(2, edges=[[0, 1]], C0=[], C1=[0], C2=[1])
edge_02 = CLGT(2, edges=[[0, 1]], C0=[0], C1=[], C2=[1])
point0 = CLGT(1, C0 = [0])
point1 = CLGT(1, C1 = [0])
point2 = CLGT(1, C2 = [0])
positives = [
    edge_12 - edge_01, 
    edge_12 - edge_02,
    edge_01 + edge_02 + edge_12 - 1/8,
    point0 - 1/4, 
    point1 - 1/4, 
    point2 - 1/4
]
M = CLGT(2, edges=[], C0=[], C1=[0], C2=[1]) 
B = sum([
    CLGT(2, edges=[[0, 1]], C0=[0], C1=[1], C2=[]),
    CLGT(2, edges=[[0, 1]], C0=[0], C1=[], C2=[1]),
    CLGT(2, edges=[[0, 1]], C0=[], C1=[0, 1], C2=[]),
    CLGT(2, edges=[[0, 1]], C0=[], C1=[], C2=[0, 1])
])

CLGT.verify("certificates/proposition_3_4_alter")

Checking X matrices


43it [00:00, 1484.54it/s]


Solution matrices are all positive semidefinite, linear coefficients are all non-negative
Calculating multiplication tables


43it [00:00, 422.73it/s]


Done calculating linear constraints
Calculating the bound provided by the certificate


43it [00:04, 10.71it/s]

The solution is valid, it proves the bound 0


0

In [5]:
cert = pickle.load(open("certificates/proposition_3_4_alter.pickle", "rb"))

In [10]:
-27/100 + 6101/81920 + 82567/409600
#cert["slack vector"][:10]

31/5120

In [9]:
cert["slack vector"][7]

Fraction(82567, 409600)

In [5]:
cert.keys()

dict_keys(['result', 'X matrices', 'e vector', 'slack vector', 'phi vectors', 'base flags', 'typed flags', 'target', 'positives', 'factor flags', 'maximize', 'target size'])

In [3]:
len(cert["positives"])

525

In [11]:
cert["target"][7]

Fraction(-27, 100)

In [30]:
sc = verify_certificate(CLGT, cert)

e values are good
Checking X matrices


43it [00:00, 1700.02it/s]


Solution matrices are all positive semidefinite, linear coefficients are all non-negative
Calculating multiplication tables


43it [00:00, 1042.20it/s]


Done calculating linear constraints
Calculating the bound provided by the certificate


43it [00:03, 11.87it/s]


In [34]:
grs = len(list(sc.values())[0])
stt = ""
matvs = []
for kk in sc.keys():
    stt += kk + "\t"
    matvs.append([QQ(xx) for xx in sc[kk]])
print(stt)
mat = matrix(matvs)
print(mat.str())

target	positives	type0	type1	type2	type3	type4	type5	type6	type7	type8	type9	type10	type11	type12	type13	type14	type15	type16	type17	type18	type19	type20	type21	type22	type23	type24	type25	type26	type27	type28	type29	type30	type31	type32	type33	type34	type35	type36	type37	type38	type39	type40	type41	type42	
[           0         9/25        27/50        27/50         9/25            0            0       27/100         9/25       27/100            0            0         9/50         9/50            0            0        9/100            0            0            0            0        -1/10        13/50       27/100        11/25         9/20        11/25         9/20        11/25        11/25        13/50       27/100        -1/10        -1/10        -1/10       17/100       17/100         9/50       17/100        13/50       27/100        13/50        13/50        13/50       17/100       17/100       17/100         9/50        -1/10        -1/10        -1/10        -1/10            0  

In [40]:
sum(mat[:, 7])

(138617/409600)

In [44]:
1/2560 + 1/320 - 49/5120 + 6101/8192 - 27/100

95989/204800

In [39]:
print(mat[:, 7].str())

[    27/100]
[6101/81920]
[         0]
[  -49/5120]
[         0]
[         0]
[    1/2560]
[         0]
[         0]
[         0]
[     1/320]
[         0]
[         0]
[         0]
[         0]
[         0]
[         0]
[         0]
[         0]
[         0]
[         0]
[         0]
[         0]
[         0]
[         0]
[         0]
[         0]
[         0]
[         0]
[         0]
[         0]
[         0]
[         0]
[         0]
[         0]
[         0]
[         0]
[         0]
[         0]
[         0]
[         0]
[         0]
[         0]
[         0]
[         0]


In [29]:
from sage.algebras.combinatorial_theory import _unflatten_matrix, _flatten_matrix
def verify_certificate(self, file_or_cert, target_element=None, 
                           target_size=None, maximize=True, positives=None, 
                           **params):
    if isinstance(file_or_cert, str):
        file = file_or_cert
        if not file.endswith(".pickle"):
            file += ".pickle"
        with open(file, 'rb') as file:
            certificate = pickle.load(file)
    else:
        certificate = file_or_cert
    
    from fractions import Fraction
    def to_sage(dim, data):
        if dim==0:
            if isinstance(data, Fraction):
                return QQ(data)
            if isinstance(data, float):
                return RR(data)
            return data
        return [to_sage(dim-1, xx) for xx in data]

    #
    # Checking eigenvalues and positivity constraints
    #
    
    e_values = to_sage(1, certificate["e vector"])

    if len(e_values)>0 and min(e_values)<0:
        print("Solution is not valid!")
        self.fprint("Linear constraint's coefficient is negative ", 
              min(e_values))
        return -1
    print("e values are good")
    
    X_flats = to_sage(2, certificate["X matrices"])
    self.fprint("Checking X matrices")
    if self._printlevel > 0:
        iterator = tqdm(enumerate(X_flats))
    else:
        iterator = enumerate(X_flats)
    for ii, Xf in iterator:
        X = matrix(_unflatten_matrix(Xf)[0])
        try:
            if not (X.is_positive_semidefinite()):
                print("Solution is not valid!")
                self.fprint("Matrix {} is not semidefinite".format(ii))
                return -1
        except:
            RFF = RealField(prec=100)
            if not matrix(RFF, X).is_positive_semidefinite():
                print("Solution is not valid!")
                self.fprint("Matrix {} is not semidefinite".format(ii))
                return -1

    self.fprint("Solution matrices are all positive semidefinite, " + 
          "linear coefficients are all non-negative")

    #
    # Initial setup
    #

    if target_size==None:
        if "target size" in certificate:
            target_size = to_sage(0, certificate["target size"])
        else:
            raise ValueError("Target size must be specified "+
                             "if it is not part of the certificate!")
    maximize = maximize and certificate.get("maximize", True)
    mult = -1 if maximize else 1
    base_flags = certificate["base flags"]
    one_vector = vector([1]*len(base_flags))
    if target_element==None:
        if "target" in certificate:
            target_vector_exact = vector(to_sage(1, certificate["target"]))
        else:
            raise ValueError("Target must be specified "+
                             "if it is not part of the certificate!")
    else:
        target_vector_exact = (
            target_element.project()<<\
                (target_size - target_element.size())
                ).values()
        if not (target_element.ftype().afae()==1):
            one_vector = (
                target_element.ftype().project()<<\
                    (target_size - target_element.ftype().size())
                    ).values()
    ftype_data = list(certificate["typed flags"].keys())
    
    #
    # Create the semidefinite matrix data
    #

    table_list = []
    self.fprint("Calculating multiplication tables")
    if self._printlevel > 0:
        iterator = tqdm(enumerate(ftype_data))
    else:
        iterator = enumerate(ftype_data)
    for ii, dat in iterator:
        ns, ftype = dat
        ftype = self._element_constructor_(ftype[0], ftype=ftype[1], **dict(ftype[2]))
        #calculate the table
        table = self.mul_project_table(
            ns, ns, ftype, ftype_inj=[], target_size=target_size
            )
        if table!=None:
            table_list.append(table)

    #
    # Create the data from linear constraints
    #

    if positives != None:
        positives_list_exact = []
        for ii, fv in enumerate(positives):
            if isinstance(fv, BuiltFlag) or isinstance(fv, ExoticFlag):
                continue
            nf = fv.size()
            df = target_size + fv.ftype().size() - nf
            mult_table = self.mul_project_table(
                nf, df, fv.ftype(), ftype_inj=[], target_size=target_size
                )
            fvvals = fv.values()
            m = matrix([vector(fvvals*mat) for mat in mult_table])
            positives_list_exact += list(m.T)
            self.fprint("Done with positivity constraint {}".format(ii))
        positives_matrix_exact = matrix(
            len(positives_list_exact), len(base_flags), 
            positives_list_exact
            )
        e_values = vector(e_values[:len(positives_list_exact)])
    else:
        if "positives" in certificate:
            posls = to_sage(2, certificate["positives"])[:-2]
            positives_matrix_exact = matrix(len(posls), len(base_flags), posls)
            e_values = vector(e_values[:len(posls)])
        else:
            positives_matrix_exact = matrix(0, len(base_flags), [])
            e_values = vector(QQ, [])

    self.fprint("Done calculating linear constraints")

    #
    # Calculate the bound the solution provides
    #
    
    self.fprint("Calculating the bound provided by the certificate")

    slack_coeffs = {}
    slack_coeffs["target"] = - target_vector_exact
    slack_coeffs["positives"] = positives_matrix_exact.T*e_values
    
    slacks = target_vector_exact - positives_matrix_exact.T*e_values
    if self._printlevel > 0:
        iterator = tqdm(enumerate(table_list))
    else:
        iterator = enumerate(table_list)
    for ii, table in iterator:
        slvecdel = []
        for mat_gg in table:
            mat_flat = vector(
                _flatten_matrix(mat_gg.rows(), doubled=True)
                )
            slvecdel.append(mat_flat * vector(X_flats[ii]))
        slack_coeffs["type"+str(ii)] = vector(slvecdel)
        slacks -= vector(slvecdel)
    
    return slack_coeffs